# Test saved models with real-time camera test.

In [ ]:
# Importing Libs

import cv2
import numpy as np
import tensorflow as tf
from keras.models import load_model
import math
from cvzone.HandTrackingModule import HandDetector

print("TensorFlow version:", tf.__version__)

In [ ]:
# Load the Indian Sign Language model
# model = load_model('isl_model.h5')
isl_model = load_model('isl_model.h5')

isl_model.summary()


In [ ]:
from sklearn.preprocessing import LabelEncoder

class_names = [chr(i + 65) for i in range(26)]

detector = HandDetector(detectionCon=0.8, maxHands=2)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(class_names)

# PreProcess the image before classification
def preprocess_image(frame):

    hands, _ = detector.findHands(frame)

    if len(hands) == 0:
        return None

    lmlist = hands[0]['lmList']
    center = hands[0]['center']
    bbox = hands[0]['bbox']
    _, _, W, H = bbox

    data = tuple((x - center[0], y - center[1]) for x, y, _ in lmlist)

    min_x = min(tuple(x for x, _ in data))
    min_y = min(tuple(y for _, y in data))

    data = tuple((x - min_x, y - min_y) for x, y in data)

    # Note: Instead of dividing x by w and y by H, an alterative
    #       approach could be divide x and y by sqrt(w*w + h*h) [TO TRY]
    data = tuple((round((x / W), 3) , round((y / H), 3)) for x, y in data)

    return np.array(tuple(val for pair in data for val in pair))


# Makes prediction from the data using provided model
def make_prediction(model, data):

    prediction = model.predict(data[np.newaxis, ...])
    prediction_p = tf.nn.softmax(prediction)
    yhat = np.argmax(prediction_p)

    return str(label_encoder.inverse_transform([yhat]))


In [ ]:
import os


# Function to predict the sign from webcam input
def predict_sign(model, image_preprocessor):

    # Open the webcam
    cap = cv2.VideoCapture(0)

    while True:
        # Read the frame from the webcam
        ret, frame = cap.read()

        if ret is False:
            raise('Error in Frame Capture!')

        # Generate data from frame to be paased from model input
        data = image_preprocessor(frame)

        if data is None:
            sign = 'XXX'
        else:
            # Make prediction using the model and data from the frame
            sign = make_prediction(model, data)

        # Display the predicted sign label on the screen
        cv2.putText(frame, sign, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Indian Sign Language Recognition', frame)

        # Stop the program when the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close all windows
    cap.release()
    cv2.destroyAllWindows()

# Call the function to start predicting signs from webcam input
predict_sign(model=isl_model, image_preprocessor=preprocess_image)
